<a href="https://colab.research.google.com/github/dit21dsc/Image-coloring/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import keras
from tensorflow.keras.utils  import array_to_img, img_to_array, load_img
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Sequential, Model
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
#OPTIMIZER
from tensorflow.keras.optimizers import RMSprop
import random

In [3]:
from keras.layers import Conv2D,BatchNormalization , ReLU , Conv2DTranspose

In [4]:
Gray_NPY = np.load("/content/drive/MyDrive/gray_scale.npy")
AB_1_NPY = np.load("/content/drive/MyDrive/ab1.npy")
AB_2_NPY = np.load("/content/drive/MyDrive/ab2.npy")
AB_3_NPY = np.load("/content/drive/MyDrive/ab3.npy")
AB_NPY = np.concatenate([AB_1_NPY, AB_2_NPY, AB_3_NPY], axis=0)



In [5]:
print(type(Gray_NPY))
print(type(AB_NPY))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [6]:
print(Gray_NPY.shape)
print(AB_NPY.shape)

(25000, 224, 224)
(25000, 224, 224, 2)


In [7]:
start_split = 2000
end_split = 2500
end_non_seen= 3000
dataset_count = 500

In [8]:
X_Train = Gray_NPY[start_split:end_split,:,:].astype("float32").reshape(dataset_count,Gray_NPY.shape[1],Gray_NPY.shape[2])

In [ ]:
X_Non_Seen = Gray_NPY[end_split:end_non_seen,:,:].astype("float32").reshape(dataset_count,Gray_NPY.shape[1],Gray_NPY.shape[2])

In [ ]:
Y_Train = AB_NPY[start_split:end_split,:,:].astype("float32")

In [ ]:
print(X_Train.shape)
print(X_Non_Seen.shape)
print(Y_Train.shape)

In [ ]:
def line_image(gray_images,splitting_count=dataset_count,preprocess_function=preprocess_input):
    Zeros_Imp = np.zeros((splitting_count,224,224,3))
    for indexing in range(0,3):
        Zeros_Imp[:splitting_count,:,:,indexing] = gray_images[:splitting_count]
        
    return preprocess_function(Zeros_Imp)

In [ ]:
Input_Images = line_image(X_Train,dataset_count)

Non_Seen_Input = line_image(X_Non_Seen,dataset_count)



In [ ]:
print(Input_Images.shape)
print(Non_Seen_Input.shape)

In [ ]:
def from_lab_to_rgb(gray_images,ab_images,n=10):
    Zeros_Imp = np.zeros((n,224,224,3))
    
    Zeros_Imp[:,:,:,0] = gray_images[0:n:]
    Zeros_Imp[:,:,:,1:] = ab_images[0:n:]
    
    Zeros_Imp = Zeros_Imp.astype("uint8")
    
    Main_Img = []
    
    for indexing in range(0,n):
        Main_Img.append(cv2.cvtColor(Zeros_Imp[indexing],cv2.COLOR_LAB2RGB))
        
    Main_Img = np.array(Main_Img)
    
    return Main_Img

In [ ]:
Output_Images = preprocess_input(from_lab_to_rgb(X_Train,Y_Train,n=dataset_count))

print(Output_Images.shape)

In [ ]:
Gray_Zeros_Imp = np.zeros((dataset_count,Gray_NPY.shape[1],Gray_NPY.shape[2],1))
Gray_Zeros_Imp[:,:,:,0] = X_Train
print(Gray_Zeros_Imp.shape)

In [ ]:
compile_loss = "mse"
compile_optimizer = RMSprop(learning_rate=0.0001,decay=1e-8)
compile_metrics = ["accuracy"]
input_dim = (Input_Images.shape[1],Input_Images.shape[2],Input_Images.shape[3])
output_class = 1

In [ ]:
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")

In [ ]:
Encoder_G = Sequential()
Encoder_G.add(Conv2D(32,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Encoder_G.add(BatchNormalization())
Encoder_G.add(ReLU())

In [ ]:
Encoder_G.add(Conv2D(64,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Encoder_G.add(BatchNormalization())
Encoder_G.add(ReLU())
#

In [ ]:
Encoder_G.add(Conv2D(128,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Encoder_G.add(BatchNormalization())
Encoder_G.add(ReLU())


In [ ]:

Encoder_G.add(Conv2D(256,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Encoder_G.add(BatchNormalization())
Encoder_G.add(ReLU())

In [ ]:
Decoder_G = Sequential()
Decoder_G.add(Conv2DTranspose(128,(2,2),padding = "same",use_bias = True))
Decoder_G.add(ReLU())
#

In [ ]:
Decoder_G.add(Conv2DTranspose(64,(2,2),padding = "same",use_bias = True))
Decoder_G.add(ReLU())
#

In [ ]:
Decoder_G.add(Conv2DTranspose(32,(2,2),padding = "same",use_bias = True))
Decoder_G.add(ReLU())
#

In [ ]:
Decoder_G.add(Conv2DTranspose(3,(2,2),padding = "same",use_bias = True))
Decoder_G.add(ReLU())

In [ ]:
Auto_Encoder = Sequential([Encoder_G,Decoder_G])
Auto_Encoder.compile(loss=compile_loss,optimizer=compile_optimizer,metrics=compile_metrics)
Auto_Encoder_Model = Auto_Encoder.fit(Input_Images,Output_Images,epochs=25,callbacks=[Early_Stopper,Checkpoint_Model],batch_size=8)

In [ ]:
#PREDICTION

Prediction_IMG = Auto_Encoder.predict(Input_Images[:100])

Prediction_Non_Seen = Auto_Encoder.predict(Non_Seen_Input[:100])

print(Prediction_IMG[5].shape)
print(Prediction_Non_Seen[5].shape)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(20,20))
prediction_img_number = random.random.randint(0,100)

Original_Img = Input_Images[prediction_img_number]
Predict_Image_AE = Prediction_IMG[prediction_img_number]
axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("INPUT")
axis[1].imshow(Predict_Image_AE)
axis[1].set_xlabel(Predict_Image_AE.shape)
axis[1].set_ylabel(Predict_Image_AE.size)
axis[1].set_title("AUTO ENCODER OUTPUT")

#Text(0.5, 1.0, 'AUTO ENCODER OUTPUT')